In [15]:
import pandas as pd
from dsgd.DSClassifierMultiQ import DSClassifierMultiQ


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Clasificadores
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

accuracys = {}

In [16]:
data = pd.read_csv('data/nasa.csv')
data.drop(["Neo Reference ID", "Name", "Close Approach Date", "Orbit Determination Date" ,"Orbiting Body", "Equinox"] , axis=1, inplace=True)
print("tamaño del dataset: ", data.shape)
print(data.dtypes)

tamaño del dataset:  (4687, 34)
Absolute Magnitude              float64
Est Dia in KM(min)              float64
Est Dia in KM(max)              float64
Est Dia in M(min)               float64
Est Dia in M(max)               float64
Est Dia in Miles(min)           float64
Est Dia in Miles(max)           float64
Est Dia in Feet(min)            float64
Est Dia in Feet(max)            float64
Epoch Date Close Approach         int64
Relative Velocity km per sec    float64
Relative Velocity km per hr     float64
Miles per hour                  float64
Miss Dist.(Astronomical)        float64
Miss Dist.(lunar)               float64
Miss Dist.(kilometers)          float64
Miss Dist.(miles)               float64
Orbit ID                          int64
Orbit Uncertainity                int64
Minimum Orbit Intersection      float64
Jupiter Tisserand Invariant     float64
Epoch Osculation                float64
Eccentricity                    float64
Semi Major Axis                 float64
Inclinat

In [17]:
print(data.head(1))

   Absolute Magnitude  Est Dia in KM(min)  Est Dia in KM(max)  \
0                21.6             0.12722            0.284472   

   Est Dia in M(min)  Est Dia in M(max)  Est Dia in Miles(min)  \
0         127.219879         284.472297               0.079051   

   Est Dia in Miles(max)  Est Dia in Feet(min)  Est Dia in Feet(max)  \
0               0.176763            417.388066            933.308089   

   Epoch Date Close Approach  ...  Inclination  Asc Node Longitude  \
0               788947200000  ...     6.025981          314.373913   

   Orbital Period  Perihelion Distance  Perihelion Arg  Aphelion Dist  \
0      609.599786             0.808259        57.25747       2.005764   

   Perihelion Time  Mean Anomaly  Mean Motion  Hazardous  
0     2.458162e+06    264.837533     0.590551       True  

[1 rows x 34 columns]


In [18]:
data = data.sample(frac=1).reset_index(drop=True)
data["Hazardous"] = data["Hazardous"].apply(lambda x: 1 if x == True else 0)
# Convert everithing to numeric
data = data.apply(pd.to_numeric)
cut = int(0.3 * len(data))

# Separate the trainig and tesing sets
X_train = data.iloc[:cut, :-1].values
y_train = data.iloc[:cut, -1].values
X_test = data.iloc[cut:, :-1].values
y_test = data.iloc[cut:, -1].values

In [19]:
DSC = DSClassifierMultiQ(2, min_iter=50, max_iter=400, debug_mode=True, \
                         lossfn="MSE", num_workers=0, min_dloss=1e-7, precompute_rules=True)
DSC2 = DSClassifierMultiQ(2, min_iter=50, max_iter=400, debug_mode=True, \
                         lossfn="MSE", num_workers=0, min_dloss=1e-7, precompute_rules=True)

In [20]:
losses, epoch, dt = DSC.fit(X_train, y_train, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=data.columns[:-1], print_every_epochs=31, print_final_model=True)

Optimization started
Processing epoch	373	0.0411	
Training time: 124.67s, epochs: 400

Least training loss reached: 0.040
DSModelMultiQ(
  DS Classifier using 132 rules
  
  Rule 1: Absolute Magnitude < 20.313
  	C1: 0.000	C2: 0.681	Unc: 0.319
  
  Rule 2: 20.313 < Absolute Magnitude < 22.281
  	C1: 0.000	C2: 0.871	Unc: 0.129
  
  Rule 3: 22.281 < Absolute Magnitude < 24.248
  	C1: 0.865	C2: 0.000	Unc: 0.135
  
  Rule 4: Absolute Magnitude > 24.248
  	C1: 0.726	C2: 0.000	Unc: 0.274
  
  Rule 5: Est Dia in KM(min) < 0.023
  	C1: 0.335	C2: 0.000	Unc: 0.665
  
  Rule 6: 0.023 < Est Dia in KM(min) < 0.197
  	C1: 0.113	C2: 0.074	Unc: 0.813
  
  Rule 7: 0.197 < Est Dia in KM(min) < 0.372
  	C1: 0.077	C2: 0.103	Unc: 0.820
  
  Rule 8: Est Dia in KM(min) > 0.372
  	C1: 0.087	C2: 0.098	Unc: 0.815
  
  Rule 9: Est Dia in KM(max) < 0.051
  	C1: 0.262	C2: 0.000	Unc: 0.738
  
  Rule 10: 0.051 < Est Dia in KM(max) < 0.441
  	C1: 0.237	C2: 0.000	Unc: 0.763
  
  Rule 11: 0.441 < Est Dia in KM(max) < 0

In [21]:
losses, epoch, dt = DSC2.fit(X_train, y_train, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=True,
                                column_names=data.columns[:-1], print_every_epochs=31, print_final_model=True)

Optimization started
Processing epoch	32	0.2500	
Training time: 23.89s, epochs: 52

Least training loss reached: 0.250
DSModelMultiQ(
  DS Classifier using 1188 rules
  
  Rule 1: Absolute Magnitude < 20.313
  	C1: 0.156	C2: 0.044	Unc: 0.800
  
  Rule 2: 20.313 < Absolute Magnitude < 22.281
  	C1: 0.073	C2: 0.126	Unc: 0.800
  
  Rule 3: 22.281 < Absolute Magnitude < 24.248
  	C1: 0.110	C2: 0.090	Unc: 0.800
  
  Rule 4: Absolute Magnitude > 24.248
  	C1: 0.124	C2: 0.076	Unc: 0.800
  
  Rule 5: Est Dia in KM(min) < 0.023
  	C1: 0.114	C2: 0.086	Unc: 0.800
  
  Rule 6: 0.023 < Est Dia in KM(min) < 0.197
  	C1: 0.114	C2: 0.086	Unc: 0.801
  
  Rule 7: 0.197 < Est Dia in KM(min) < 0.372
  	C1: 0.054	C2: 0.146	Unc: 0.800
  
  Rule 8: Est Dia in KM(min) > 0.372
  	C1: 0.189	C2: 0.011	Unc: 0.800
  
  Rule 9: Est Dia in KM(max) < 0.051
  	C1: 0.198	C2: 0.002	Unc: 0.800
  
  Rule 10: 0.051 < Est Dia in KM(max) < 0.441
  	C1: 0.096	C2: 0.103	Unc: 0.801
  
  Rule 11: 0.441 < Est Dia in KM(max) < 0.8

In [22]:
# Using the predict method, you can predict the classes of new records
# In this case we will predict the testing set
y_pred = DSC.predict(X_test)
y_pred2 = DSC2.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred2))
accuracys["DSClassifierMultiQ"] = accuracy_score(y_test, y_pred)
accuracys["DSClassifierMultiQ2"] = accuracy_score(y_test, y_pred2)

              precision    recall  f1-score   support

           0       0.96      0.95      0.96      2759
           1       0.75      0.81      0.78       522

    accuracy                           0.93      3281
   macro avg       0.86      0.88      0.87      3281
weighted avg       0.93      0.93      0.93      3281

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      2759
           1       0.00      0.00      0.00       522

    accuracy                           0.84      3281
   macro avg       0.42      0.50      0.46      3281
weighted avg       0.71      0.84      0.77      3281



/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [24]:
print(confusion_matrix(y_test, y_pred,))
print(confusion_matrix(y_test, y_pred2,))

[[2622  137]
 [ 101  421]]
[[2759    0]
 [ 522    0]]


In [25]:
# Global interpretability
DSC.print_most_important_rules(classes=["no hazzardous", "hazzardous"])



Most important rules for class no hazzardous

	[0.868] R78: 0.083 < Minimum Orbit Intersection < 0.145
			no : 0.868	haz: 0.000	Unc: 0.132

	[0.865] R2: 22.281 < Absolute Magnitude < 24.248
			no : 0.865	haz: 0.000	Unc: 0.135

	[0.833] R79: Minimum Orbit Intersection > 0.145
			no : 0.833	haz: 0.000	Unc: 0.167

	[0.726] R3: Absolute Magnitude > 24.248
			no : 0.726	haz: 0.000	Unc: 0.274

	[0.586] R111: Perihelion Distance > 0.978
			no : 0.586	haz: 0.000	Unc: 0.414

	[0.532] R85: 2457036.785 < Epoch Osculation < 2457699.651
			no : 0.532	haz: 0.000	Unc: 0.468

	[0.531] R123: Perihelion Time > 2458383.603
			no : 0.531	haz: 0.000	Unc: 0.469

	[0.531] R75: Orbit Uncertainity > 5.581
			no : 0.531	haz: 0.000	Unc: 0.469

	[0.508] R113: 111.888 < Perihelion Arg < 182.398
			no : 0.508	haz: 0.000	Unc: 0.492

	[0.484] R98: 13.609 < Inclination < 21.182
			no : 0.484	haz: 0.000	Unc: 0.516

	[0.457] R68: Orbit ID < 3.608
			no : 0.457	haz: 0.000	Unc: 0.543

	[0.449] R81: 4.197 < Jupiter Tisse

In [26]:
X = data.drop('Hazardous', axis=1)  # Asumiendo que 'target' es la columna objetivo
y = data['Hazardous']

X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


In [27]:
# Diccionario de modelos
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(max_depth=5),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# Función para entrenar y evaluar modelos
def train_and_evaluate(models, X_train, y_train, X_test, y_test):
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracys[name] = accuracy
        print(f'{name} Accuracy: {accuracy:.4f}')
        print(f'Classification Report for {name}:\n{classification_report(y_test, y_pred)}')
        print(f'Confusion Matrix for {name}:\n{confusion_matrix(y_test, y_pred)}\n')

# Llamada a la función
train_and_evaluate(models, X1_train, y1_train, X1_test, y1_test)


Logistic Regression Accuracy: 0.8387
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       983
           1       0.00      0.00      0.00       189

    accuracy                           0.84      1172
   macro avg       0.42      0.50      0.46      1172
weighted avg       0.70      0.84      0.77      1172

Confusion Matrix for Logistic Regression:
[[983   0]
 [189   0]]



/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Random Forest Accuracy: 0.9940
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       983
           1       1.00      0.96      0.98       189

    accuracy                           0.99      1172
   macro avg       1.00      0.98      0.99      1172
weighted avg       0.99      0.99      0.99      1172

Confusion Matrix for Random Forest:
[[983   0]
 [  7 182]]

Support Vector Machine Accuracy: 0.8387
Classification Report for Support Vector Machine:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       983
           1       0.00      0.00      0.00       189

    accuracy                           0.84      1172
   macro avg       0.42      0.50      0.46      1172
weighted avg       0.70      0.84      0.77      1172

Confusion Matrix for Support Vector Machine:
[[983   0]
 [189   0]]

K-Nearest Neighbors Accuracy: 0.8123
Classification Rep

/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dani/Escritorio/Trabajo dirigido/python-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [28]:
for key in accuracys:
    print(key, accuracys[key])

DSClassifierMultiQ 0.927461139896373
DSClassifierMultiQ2 0.8409021639743981
Logistic Regression 0.8387372013651877
Random Forest 0.9940273037542662
Support Vector Machine 0.8387372013651877
K-Nearest Neighbors 0.8122866894197952
